In [1]:
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Load simple dataset

We load the data from the UCR TS archive, you can get the data at http://www.cs.ucr.edu/~eamonn/time_series_data/

In [2]:
datapath = '/media/christiaan/extra/timeseries/UCR_TS_Archive_2015'

In [3]:
datasets_train = {}
datasets_test = {}
for i in ["X", "Y", "Z"]:
    path_to_data_train = datapath + '/uWaveGestureLibrary_'+i+'/uWaveGestureLibrary_'+i+'_TRAIN'
    path_to_data_test = datapath + '/uWaveGestureLibrary_'+i+'/uWaveGestureLibrary_'+i+'_TEST'
    datasets_train[i] = np.genfromtxt(path_to_data_train, delimiter=',')
    datasets_test[i] = np.genfromtxt(path_to_data_test, delimiter=',')
    print(datasets_train[i].shape, datasets_test[i].shape)

(896, 316) (3582, 316)
(896, 316) (3582, 316)
(896, 316) (3582, 316)


In [4]:
y_train = np.vstack((datasets_train['X'][:,0], datasets_train['Y'][:,0], datasets_train['Z'][:,0])).transpose()
#Check labels are the same across channels
print(y_train.std(axis=1).sum())
y_train = np.array(y_train[:,0], dtype='int')
print(y_train.shape)

y_test = np.array(datasets_test['X'][:,0], dtype='int')
print(y_test.shape)

0.0
(896,)
(3582,)


In [5]:
X_train  = np.stack((datasets_train['X'][:,1:], datasets_train['Y'][:,1:], datasets_train['Z'][:,1:]), axis=-1)
print(X_train.shape)
X_test  = np.stack((datasets_test['X'][:,1:], datasets_test['Y'][:,1:], datasets_test['Z'][:,1:]), axis=-1)
print(X_test.shape)

(896, 315, 3)
(3582, 315, 3)


In [6]:
X_train[:3,:4,:]

array([[[-0.30424, -2.1194 , -1.529  ],
        [-0.30424, -2.1194 , -1.529  ],
        [-0.30424, -2.1194 , -1.529  ],
        [-0.30424, -2.1194 , -1.529  ]],

       [[ 1.6273 ,  0.66662,  1.7869 ],
        [ 1.6273 ,  0.66662,  1.7869 ],
        [ 1.6273 ,  0.66662,  1.7869 ],
        [ 1.6273 ,  0.66662,  1.7869 ]],

       [[ 0.66128, -0.18973,  0.52125],
        [ 0.66128, -0.18973,  0.52125],
        [ 0.66128, -0.18973,  0.52125],
        [ 0.66128, -0.18973,  0.52125]]])

In [7]:
#Change class labels ranging from 0 to n-1
classlabels = list(set(y_train))
mapclasses = {classlabels[i] : i for i in range(len(classlabels))}
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')

In [8]:
ntrain = X_train.shape[0]
num_training = int(ntrain * 0.7)
num_validation = ntrain - num_training
num_test = X_test.shape[0]

#First sort the data in random order
np.random.seed(123)
neworder = np.random.permutation(ntrain)
X_train_random = X_train[neworder,:]
y_train_random = y_train[neworder]

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = X_train_random[mask]
y_val = y_train_random[mask]
mask = range(num_training)
X_train = X_train_random[mask]
y_train = y_train_random[mask]


print(X_val.shape)

(269, 315, 3)


In [9]:
# We need to convert the output
from keras.utils.np_utils import to_categorical
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)
print(y_train_binary[:10,:])

Using Theano backend.


[[ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]]


## Build model architecture

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam

In [22]:
def generate_model(x_shape, class_number, filters, fc_hidden, lr = 0.01):
    dim_length = x_shape[1]
    dim_channels = x_shape[2]
    outputdim = class_number
    
    model = Sequential()
    # TODO: weight initialization (in layer constructor)
    # TODO: regularation etc
    model.add(Convolution1D(filters[0], 3, border_mode='same', input_shape=(dim_length, dim_channels)))
    for filter_number in filters[1:]:
        model.add(Convolution1D(filter_number, 3, border_mode='same'))
        model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(output_dim=fc_hidden)) # Fully connected layer
    model.add(Activation('relu')) # Relu activation
    model.add(Dense(output_dim=outputdim))
    model.add(Activation("softmax")) # Final classification layer    
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(lr=lr), 
                  metrics=['accuracy'])
    
    return model


In [31]:
def generate_models(x_shape, number_of_classes, number_of_models = 5):
    models = []    
    for _ in range(0, number_of_models):
        hyperparameters = generate_hyperparameter_set()
        filters = hyperparameters['filters']
        fc_hidden = hyperparameters['fc_hidden_nodes']
        lr = hyperparameters['learning_rate']
        models.append((generate_model(x_shape, number_of_classes, filters, fc_hidden, lr), hyperparameters))        
    return models

In [32]:
def generate_hyperparameter_set(min_layers = 1, max_layers = 10, 
                                 min_filters = 10, max_filters = 100, 
                                 min_fc_nodes = 10, max_fc_nodes = 100):   
    number_of_layers = np.random.randint(min_layers, max_layers)
    number_of_filters = np.random.randint(min_filters, max_filters, number_of_layers)
    number_of_fc_nodes = np.random.randint(min_fc_nodes, max_fc_nodes)
    low = 1
    high = 4    
    lr = 10**(-np.random.uniform(low, high))        
    return {'filters':number_of_filters, 'fc_hidden_nodes':number_of_fc_nodes, 'learning_rate':lr}    

In [34]:
for model, hyperparameters in generate_models(X_train.shape, len(set(y_train))):
    model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution1d_162 (Convolution1D)  (None, 315, 14)     140         convolution1d_input_32[0][0]     
____________________________________________________________________________________________________
convolution1d_163 (Convolution1D)  (None, 315, 83)     3569        convolution1d_162[0][0]          
____________________________________________________________________________________________________
activation_193 (Activation)        (None, 315, 83)     0           convolution1d_163[0][0]          
____________________________________________________________________________________________________
convolution1d_164 (Convolution1D)  (None, 315, 87)     21750       activation_193[0][0]             
___________________________________________________________________________________________

In [ ]:
model = Sequential()

In [ ]:
model.add(Convolution1D(32, 3, border_mode='same', input_shape=(dim_length, dim_channels)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_length=2))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dense(output_dim=30)) # Fully connected layer
model.add(Activation('relu')) # Relu activation
model.add(Dense(output_dim=outputdim))
model.add(Activation("softmax")) # Final classification layer

In [ ]:
model.summary()

Now we configure the learning process:

In [ ]:
from keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), metrics=['accuracy'])

## Fit the model on the data

In [ ]:
model.fit(X_train, y_train_binary, nb_epoch=10)

## Validate the model on validation set

In [ ]:
loss_and_metrics = model.evaluate(X_val, y_val_binary, batch_size=32)
loss_and_metrics

In [ ]:
classes = model.predict_classes(X_val, batch_size=32)
proba = model.predict_proba(X_val, batch_size=32)

In [ ]:
print(np.hstack((proba, np.vstack((classes, y_val)).transpose())))

Let's compute some more metrics, such as the confusion matrix and the ROC curve

In [ ]:
from sklearn import metrics
metrics.confusion_matrix(classes, y_val)

## Test on testset

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test_binary, batch_size=32)
loss_and_metrics

In [ ]:
#Best score by Dynamic time warping:
1 - 0.034 